# Exercise - Multi step Workflow - STARTER

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda, RunnableSequence
from pydantic import BaseModel, Field

In [3]:
import dotenv
import os

dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)
```

In [5]:
# TODO - Instantiate your chat model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key = OPENAI_API_KEY,
)

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [6]:
logs = []

In [7]:
parser = StrOutputParser()

In [8]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [13]:
# TODO - Your prompt Template
idea_prompt = PromptTemplate(
    template=(
        """
        You are a creative business advisor.
        Generate one innovative business idea in the industry:
        {industry}.
        Provide a brief description of the idea.
        """
    )
)

In [14]:
# TODO - Create your idea_chain: idea_prompt -> llm -> parse_and_log_output_chain
idea_chain = RunnableSequence(idea_prompt, llm, parse_and_log_output_chain)

In [21]:
# TODO - Test your idea_chain invoking it by passing an industy like "agro" to it
idea_result = idea_chain.invoke({"industry" : "agro"})

In [22]:
idea_result["output"]

'**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are self-contained, modular farming units designed for urban environments and small spaces. Each pod utilizes hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT technology, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be installed in homes, schools, or community centers, promoting local food production and sustainability. Users can connect to a mobile app that provides guidance on crop selection, growth tracking, and even recipes for using their fresh produce. Additionally, the business can offer subscription services for seed kits, nutrients, and maintenance support, creating a recurring revenue stream.\n\nThis innovative approach not only addresses food security in urban ar

In [23]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are self-contained, modular farming units designed for urban environments and small spaces. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be connected to a mobile app that allows users to track growth progress, receive alerts for maintenance, and access educational resources on sustainable farming practices. Additionally, the app can facilitate community engagement by connecting users with local farmers for seed exchanges, workshops, and collaborative growing projects.\n\nThis innovative solution not only addresses food security in urban areas but also promotes sustainable

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [24]:
# TODO - Your prompt Template
analysis_prompt = PromptTemplate(
    template=(
        """
        You are professional reviewers for business ideas.
        Based on the following idea: {idea}, analyze its strengths and weaknesses.
        """
    )
)

In [25]:
# TODO - Your chain
analysis_chain = RunnableSequence(analysis_prompt, llm, parse_and_log_output_chain)

In [26]:
# TODO - Test your analysis_chain invoking it by passing idea_result["output"] to it
analysis_result = analysis_chain.invoke({"idea" : idea_result["output"]})

In [28]:
analysis_result["output"]

'### Strengths\n\n1. **Sustainability Focus**: The Smart Vertical Farming Pods align with the growing trend towards sustainability and local food production. They reduce the carbon footprint associated with transporting food and promote the use of fresh, organic produce.\n\n2. **Space Efficiency**: The modular design is ideal for urban environments where space is limited. This makes it accessible to a wide range of consumers, including apartment dwellers and schools.\n\n3. **Technology Integration**: The use of IoT technology allows for real-time monitoring and optimization of growing conditions, which can lead to higher yields and better quality produce. This tech-savvy approach appeals to a younger, more environmentally conscious demographic.\n\n4. **Educational Component**: By promoting local food production and sustainable practices, the pods can serve as educational tools in schools and community centers, fostering a culture of healthy eating and environmental awareness.\n\n5. **R

In [30]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are self-contained, modular farming units designed for urban environments and small spaces. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be connected to a mobile app that allows users to track growth progress, receive alerts for maintenance, and access educational resources on sustainable farming practices. Additionally, the app can facilitate community engagement by connecting users with local farmers for seed exchanges, workshops, and collaborative growing projects.\n\nThis innovative solution not only addresses food security in urban areas but also promotes sustainable

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [38]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [47]:
# TODO - Your prompt Template
# report_prompt = PromptTemplate(
#     template=(
#         """
#         You are a business advisor with a deep knowledge of {industry}.
#         Based on the following analysis: {analysis} and business idea description: {idea},
#         create a report describing the business idea and summarizing its strengths and weaknesses.
#         """
#     )
# )

report_prompt = PromptTemplate(
    template=(
        "Here is a business analysis: "
        "Strengths & Weaknesses: {output} "
        "Generate a structured business report."
    )
)

In [48]:
report_chain = (
    report_prompt | llm.with_structured_output(schema=AnalysisReport, method="function_calling")
)

In [52]:
# TODO - Test your report_chain invoking it by passing analysis_result["output"] to it
#report_result = report_chain.invoke({"industry" : "agro",
#                               "idea" : idea_result["output"],
#                               "analysis" : analysis_result["output"]})
report_result = report_chain.invoke({"output" : analysis_result["output"]})


In [53]:
report_result

AnalysisReport(strengths=['Sustainability Focus', 'Space Efficiency', 'Technology Integration', 'Educational Component', 'Recurring Revenue Model', 'Health Benefits'], weaknesses=['Initial Cost', 'Technical Complexity', 'Maintenance Requirements', 'Market Competition', 'Regulatory Challenges', 'Limited Crop Variety'])

## 6. End to End Chain

In [54]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [55]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [56]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [57]:
e2e_result

AnalysisReport(strengths=['Urban Relevance: The concept of Smart Vertical Farming Pods is highly relevant in urban environments where space is limited. As cities grow, the demand for local food production increases, making this idea timely and appealing.', 'Sustainability: The use of hydroponic and aeroponic systems reduces the need for soil and minimizes water usage compared to traditional farming. This aligns with the growing consumer demand for sustainable and environmentally friendly practices.', 'Technology Integration: The incorporation of IoT sensors for real-time monitoring and optimization of growing conditions enhances the efficiency and effectiveness of the farming process. This tech-savvy approach can attract a younger, more tech-oriented demographic.', "User Engagement: The mobile app not only allows users to track their plants' growth but also provides educational resources and a community marketplace. This fosters a sense of community and encourages users to engage more 

In [58]:
e2e_result.strengths

['Urban Relevance: The concept of Smart Vertical Farming Pods is highly relevant in urban environments where space is limited. As cities grow, the demand for local food production increases, making this idea timely and appealing.',
 'Sustainability: The use of hydroponic and aeroponic systems reduces the need for soil and minimizes water usage compared to traditional farming. This aligns with the growing consumer demand for sustainable and environmentally friendly practices.',
 'Technology Integration: The incorporation of IoT sensors for real-time monitoring and optimization of growing conditions enhances the efficiency and effectiveness of the farming process. This tech-savvy approach can attract a younger, more tech-oriented demographic.',
 "User Engagement: The mobile app not only allows users to track their plants' growth but also provides educational resources and a community marketplace. This fosters a sense of community and encourages users to engage more deeply with their food

In [59]:
e2e_result.weaknesses

['Initial Cost: The upfront investment for purchasing a Smart Vertical Farming Pod may be a barrier for some potential users. The cost of technology, materials, and setup could deter individuals who are not fully committed to urban farming.',
 'Technical Complexity: While the technology is a strength, it may also pose a challenge for less tech-savvy individuals. Users may struggle with setup, maintenance, or troubleshooting, which could lead to frustration and abandonment of the product.',
 'Market Competition: The vertical farming and home gardening market is becoming increasingly competitive, with various players offering similar solutions. Differentiating the product and establishing a strong brand presence will be crucial.',
 'Regulatory Challenges: Depending on the location, there may be zoning laws or regulations that could impact the installation and operation of these farming pods, particularly in densely populated urban areas.',
 'Dependency on Technology: The reliance on IoT 

In [60]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are self-contained, modular farming units designed for urban environments and small spaces. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be connected to a mobile app that allows users to track growth progress, receive alerts for maintenance, and access educational resources on sustainable farming practices. Additionally, the app can facilitate community engagement by connecting users with local farmers for seed exchanges, workshops, and collaborative growing projects.\n\nThis innovative solution not only addresses food security in urban areas but also promotes sustainable

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity